In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet50
from keras_tuner import RandomSearch

train_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\training'
valid_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\validation'
test_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\test'

last_path = [[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,1,1],[1,0,1],[1,1,1]]
prefix = [f"{i:03d}" for i in range(505, 513)]
prefix_to_label = dict(zip(prefix, last_path))

def process_dataset(root_folder):
    image_paths = []
    label_data = []
    label_data_1 = []
    label_data_2 = []
    label_data_3 = []
    for roots, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.jpg'):
                # 파일 이름 분석을 위해 숫자만 추출
                prefix = file[0:3]
                
                # 접두사에 따른 레이블 할당
                label = prefix_to_label.get(prefix)
                # 유효한 레이블이 있는 경우에만 리스트에 추가
                if label is not None:
                    image_paths.append(os.path.join(roots, file))
                    label_data_1.append(label[0])
                    label_data_2.append(label[1])
                    label_data_3.append(label[2])

    label_data.append(np.array(label_data_1))
    label_data.append(np.array(label_data_2))
    label_data.append(np.array(label_data_3))
    return image_paths, label_data

# 각각의 데이터셋에 대해 함수를 호출
train_image_paths, train_label_data = process_dataset(train_folder)
valid_image_paths, valid_label_data = process_dataset(valid_folder)
test_image_paths, test_label_data = process_dataset(test_folder)

# 필요에 따라 결과를 확인하거나 다른 처리를 수행
print(len(train_image_paths), (train_label_data))
print(len(valid_image_paths), len(valid_label_data))
print(len(test_image_paths), len(test_label_data))


7874 [array([0, 0, 0, ..., 1, 1, 1]), array([0, 0, 0, ..., 1, 1, 1]), array([0, 0, 0, ..., 1, 1, 1])]
1691 3
1687 3


In [2]:
#이미지 전처리
def resize_img(image_paths):
    images_resized = []  # 리사이즈된 이미지를 저장할 리스트
    for image_path in image_paths:
        image = cv2.imread(image_path)  # 각 이미지 경로로부터 이미지를 읽음
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 색상 변환
        image_resized = cv2.resize(image, (128, 128))  # 이미지 리사이즈
        images_resized.append(image_resized)  # 결과 리스트에 추가
    images_resized = np.array(images_resized) / 255.0  # numpy 배열로 변환 및 정규화
    return images_resized

train_image_resized = resize_img(train_image_paths)
valid_image_resized = resize_img(valid_image_paths)
test_image_resized = resize_img(test_image_paths)

print('done')


done


In [3]:
# 모델 구성    
input_tensor = Input(shape=(128, 128, 3))
base_model = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False, input_shape=(128, 128, 3))
# base_model = tf.keras.applications.InceptionV3(input_tensor=input_tensor, 
#                                                weights="imagenet", 
#                                                include_top=False, 
#                                                input_shape=(128, 128, 3))
    
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
#x = Dropout(0.1)(x)
x = Dense(512, activation='relu')(x)
#x = Dropout(0.1)(x)
x = Dense(512, activation='relu')(x)
#x = Dropout(0.1)(x)
x = Dense(512, activation='relu')(x)
#x = Dropout(0.1)(x)
x = Dense(512, activation='relu')(x)
prediction1 = Dense(1, activation='sigmoid')(x)
prediction2 = Dense(1, activation='sigmoid')(x)
prediction3 = Dense(1, activation='sigmoid')(x)
for layer in base_model.layers:
    base_model.trainable = False
for layer in base_model.layers[-9:]:
    base_model.trainable = True
# for layer in base_model.layers:
#     base_model.trainable = False
# for layer in base_model.layers[150:]:
#     base_model.trainable = True
model = Model(inputs=input_tensor, outputs=[prediction1, prediction2, prediction3])


In [4]:
earlystopping = EarlyStopping(monitor='val_loss', patience=7, mode='min', verbose=1)

model.compile(optimizer=optimizers.Adam(learning_rate=0.0002),
              loss=['binary_crossentropy','binary_crossentropy','binary_crossentropy'], loss_weights=[0.4, 0.3, 0.3],
              metrics=['accuracy'])

# 모델 훈련
model.fit(train_image_resized, 
          train_label_data, 
          validation_data=(valid_image_resized, valid_label_data), 
          epochs=25, 
          batch_size=32,
          callbacks=[earlystopping])

# 모델 평가 (테스트 데이터셋)
eval_results = model.evaluate(test_image_resized, test_label_data)
print('\n총 테스트 손실:', eval_results[0])
print('출력1 테스트 손실:', eval_results[1], '출력1 테스트 정확도:', eval_results[4])
print('출력2 테스트 손실:', eval_results[2], '출력2 테스트 정확도:', eval_results[5])
print('출력3 테스트 손실:', eval_results[3], '출력3 테스트 정확도:', eval_results[6])


Epoch 1/25
247/247 [==============================] - 36s 116ms/step - loss: 0.6483 - dense_5_loss: 0.6714 - dense_6_loss: 0.6730 - dense_7_loss: 0.5929 - dense_5_accuracy: 0.5839 - dense_6_accuracy: 0.5712 - dense_7_accuracy: 0.6779 - val_loss: 0.6959 - val_dense_5_loss: 0.6933 - val_dense_6_loss: 0.6951 - val_dense_7_loss: 0.7000 - val_dense_5_accuracy: 0.5021 - val_dense_6_accuracy: 0.5021 - val_dense_7_accuracy: 0.4997
Epoch 2/25
247/247 [==============================] - 25s 100ms/step - loss: 0.5787 - dense_5_loss: 0.6293 - dense_6_loss: 0.6320 - dense_7_loss: 0.4579 - dense_5_accuracy: 0.6356 - dense_6_accuracy: 0.6393 - dense_7_accuracy: 0.7789 - val_loss: 0.7614 - val_dense_5_loss: 0.7264 - val_dense_6_loss: 0.7040 - val_dense_7_loss: 0.8654 - val_dense_5_accuracy: 0.5021 - val_dense_6_accuracy: 0.5021 - val_dense_7_accuracy: 0.4997
Epoch 3/25
247/247 [==============================] - 25s 102ms/step - loss: 0.5289 - dense_5_loss: 0.6013 - dense_6_loss: 0.5679 - dense_7_loss: 

In [6]:
model.save(r'E:\AImodel\models\Face-ResNet-BicycleCrunch-multilabel-model')


INFO:tensorflow:Assets written to: E:\AImodel\models\Face-ResNet-BicycleCrunch-multilabel-model\assets


INFO:tensorflow:Assets written to: E:\AImodel\models\Face-ResNet-BicycleCrunch-multilabel-model\assets
